In [1]:
import pandas as pd
import numpy as np
import spacy
import scattertext as st
#import imp; imp.reload(st)
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import pickle
from nltk.corpus import reuters
%matplotlib inline  

In [2]:
nlp = spacy.en.English()

In [3]:
uci_df = pd.read_csv('data/uci-news-aggregator.csv.gz')

In [4]:
traditional_publishers = ['Forbes','Bloomberg','Los Angeles Times','TIME','Wall Street Journal']
repubable_celebrity_gossip = ['TheCelebrityCafe.com', 'PerezHilton.com']

In [5]:
real_df = uci_df[uci_df['PUBLISHER'].isin(traditional_publishers)]
real_df.columns = [x.lower() for x in real_df.columns]
real_df['type'] = 'traditional'

In [6]:
df = pd.read_csv('data/fake.csv.gz')
df = df.append(real_df)
df = df[df['title'].apply(lambda x: type(x) == str)]
df['clean_title'] = df['title'].apply(lambda x: ' '.join(x.split('»')[0].split('>>')[0].split('[')[0].split('(')[0].split('|')[0].strip().split()))
df = df.ix[df['clean_title'].drop_duplicates().index]
df['parsed_title'] = df['clean_title'].apply(nlp)

In [7]:
df['meta'] = df['author'].fillna('') + df['publisher'].fillna('') + ' ' + df['site_url'].fillna('')
df['category'] = df['type'].apply(lambda x: 'Real' if x == 'traditional' else 'Fake')

In [8]:
fake_df = df[df['category'] == 'Fake']

In [9]:
fake_df.type.value_counts()

bs            10596
conspiracy      358
bias            321
hate            246
satire          148
state           118
junksci         105
fake             19
Name: type, dtype: int64

In [17]:
def news_type_viz(fake_df, news_type, minimum_term_frequency=1, minimum_not_category_term_frequency=5, comparison_types=None, stoplist = []):
    type_df = fake_df
    if comparison_types is not None:
        type_df = type_df[type_df['type'].isin(comparison_types + [news_type])]
    type_df['news_type'] = type_df['type'].apply(lambda x: news_type if (x == news_type) else ('not ' + news_type))
    vc = type_df['clean_title'].value_counts()
    type_df = type_df[type_df['clean_title'].isin(vc[vc == 1].index)]
    corpus = (st.CorpusFromParsedDocuments(type_df, 
                                           category_col = 'news_type', 
                                           parsed_col = 'parsed_title', 
                                           feats_from_spacy_doc = st.FeatsFromSpacyDoc(strip_final_period=True))
              .build()
              .get_stoplisted_unigram_corpus_and_custom(stoplist))
    # entity_types_to_censor={'FACILITY', 'NORP', 'GPE', 'PERSON', 'WORK_OF_ART', 'LOC', 'EVENT', 'LANGUAGE', 'ORG', 'PRODUCT'}
    html = st.produce_scattertext_explorer(corpus, 
                                           category=news_type, 
                                           category_name=news_type, 
                                           not_category_name='not ' + news_type,
                                           minimum_term_frequency=minimum_term_frequency,
                                           minimum_not_category_term_frequency=minimum_not_category_term_frequency,
                                           metadata=type_df['meta'],
                                           use_full_doc=True,
                                           term_ranker=st.termranking.OncePerDocFrequencyRanker,
                                           width_in_pixels=1000)
    file_name = news_type + ".html"
    open(file_name, 'wb').write(html.encode('utf-8'))
    display(IFrame(src=file_name, width = 1200, height=1000))

In [19]:
news_type_viz(fake_df, 'hate', minimum_term_frequency=1, minimum_not_category_term_frequency=20, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren'])

In [24]:
news_type_viz(fake_df, 'bias', minimum_term_frequency=1, minimum_not_category_term_frequency=20, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren','shapiro','ben'])

In [26]:
news_type_viz(fake_df, 'satire', minimum_term_frequency=2, minimum_not_category_term_frequency=20, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren','shapiro','ben','onion'])

In [36]:
news_type_viz(fake_df, 'state', minimum_term_frequency=2, minimum_not_category_term_frequency=20, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren','shapiro','ben','onion','presstv'])

In [33]:
news_type_viz(fake_df, 'junksci', minimum_term_frequency=2, minimum_not_category_term_frequency=30, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren','shapiro','ben','onion'])

In [35]:
news_type_viz(fake_df, 'fake', minimum_term_frequency=2, minimum_not_category_term_frequency=50, comparison_types=None, 
              stoplist=['dankof','collett','duke','anglin','shoaf','slattery','farren','shapiro','ben','onion'])